# Run Functions to Add Information to Projects

In [1]:
# ! pip install nltk

In [2]:
import numpy as np
import pandas as pd
from siuba import *

from calitp_data_analysis.sql import to_snakecase

from shared_utils import geography_utils

import dla_utils

import _data_utils
import _script_utils

/opt/conda/lib/python3.9/site-packages/geopandas/_compat.py:123: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(
/opt/conda/lib/python3.9/site-packages/dask_geopandas/backends.py:13: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/l

In [3]:
pd.set_option("display.max_columns", 100)
pd.set_option('display.max_colwidth', None)

## Read in Data and function development

In [4]:
GCS_FILE_PATH  = 'gs://calitp-analytics-data/data-analyses/dla/dla-iija'

In [5]:
proj = to_snakecase(pd.read_excel(f"{GCS_FILE_PATH}/FMIS_Projects_Universe_(IIJA_Reporting)_ (3-9-23).xlsx", header=[3], skiprows=range(4,1784)))

In [6]:

# df = utils.read_data_all()
proj = _data_utils.add_new_codes(proj)
    
    ## function that adds known agency name to df 
df = _script_utils.identify_agency(proj, 'summary_recipient_defined_text_field_1_value')

In [7]:
df.sample(3)

,unnamed:_0,fmis_transaction_date,program_code,program_code_description,project_number,recipient_project_number,project_title,county_code,congressional_district,project_status_description,project_description,improvement_type,improvement_type_description,total_cost_amount,obligations_amount,summary_recipient_defined_text_field_1_value,implementing_agency_locode,implementing_agency,district,county_name,rtpa_name,mpo_name
5411,NaN,2022-08-26,Z001,NaN,P062042,0814000113S,"IN RIVERSIDE & SAN BERNARDINO COUNTIES ON ROUTE 62, IN & NEAR YUCCA VALLEY & 29 PALMS BETWEEN NORTH INDIAN CANYON DR AND UTAH TRAIL MILL & OVERLAY MA",65,Cong Dist 36,Active,"IN RIVERSIDE & SAN BERNARDINO COUNTIES ON ROUTE 62, IN & NEAR YUCCA VALLEY & 29 PALMS BETWEEN NORTH INDIAN CANYON DR AND UTAH TRAIL MILL & OVERLAY MAINLINE & SHOULDERS, UPGRADE GUARDRAIL, DRAINAGE & CURB RAMPS (TC)",16,Right of Way,254334.00,85500.00,S SCAG,NaN,California,8.0,Riverside County,NaN,NaN
1629,NaN,2022-03-25,ZS3E,NaN,P018063,0813000140S,IN SAN BERNARDINO COUNTY IN VICTORVILLE FROM AMARGOSA ROAD TO COBALT ROAD WIDEN ROADBED AND CONSTRUCT MEDIAN CURB,71,Cong Dist 8,Active,ON STATE ROUTE: 18. IN SAN BERNARDINO COUNTY IN VICTORVILLE FROM AMARGOSA ROAD TO COBALT ROAD WIDEN ROADBED AND CONSTRUCT MEDIAN CURB,17,Construction Engineering,1555000.00,1399500.00,S SCAG,NaN,California,8.0,San Bernardino County,NaN,NaN
8106,NaN,2022-11-23,ZS30,NaN,X081029,0415000086S,"IN SAN MATEO COUNTY, ON ROUTES 82 AND 84 AT VARIOUS LOCATIONS. INSTALL CROSSWALK SAFETY ENHANCEMENTS (TC)",75,Cong Dist 14,Closed,"ON STATE ROUTE: 82, 84. IN SAN MATEO COUNTY, ON ROUTES 82 AND 84 AT VARIOUS LOCATIONS. INSTALL CROSSWALK SAFETY ENHANCEMENTS (TC)",21,Safety,684.38,661.17,S MTC,NaN,California,4.0,San Francisco County,NaN,NaN


## Test & Export

In [8]:
###test agg. PASS df with title column, concat and 552 rows
##throwingerror
agg = _script_utils.get_clean_data(df, full_or_agg = 'agg')

In [9]:
agg.sample(10)

,fmis_transaction_date,project_number,implementing_agency,summary_recipient_defined_text_field_1_value,program_code,program_code_description,recipient_project_number,improvement_type,improvement_type_description,project_title,obligations_amount,congressional_district,district,county_code,county_name,county_name_title,implementing_agency_locode,rtpa_name,mpo_name,project_title_new,new_description_col
3048,2022-08-26,P089112,California,S TAHOE,L25E,nan,0300000224S,17,Construction Engineering,NEAR SOUTH LAKE TAHOE FROM CASCADE ROAD TO UPPER EMERALD BAY ROAD STORM WATER QUALITY IMPROVEMENTS - (TC),1572900.00,4,3.0,17,El Dorado County,El Dorado County,nan,None,None,Construction Engineering Projects in El Dorado County,"This project is part of the nan Program, and recieved $1572900.0. This project will Construction Engineering Projects in El Dorado County."
0,2022-01-18,0151240,California,S SCAG,Z001,nan,0812000093S,6|17,4R - Restoration & Rehabilitation|Construction Engineering,IN SAN BERNARDINO CO FR 0.3 MI N/O N. ALRAY OVERHEAD TO 0.3 MI S/O OAK HILL RD OH INSTALL VEGETATION CONTROL UNDERNEATH METAL BEAM GUARD RAIL (TC),1202300.56,8,8.0,71,San Bernardino County,San Bernardino County,nan,None,None,Install Overhead in San Bernardino County,"This project is part of the nan Program, and recieved $1202300.56. This project will Install Overhead in San Bernardino County."
2230,2022-07-01,0010177,California,S STANCOG,Z45E,nan,0021000338S,18,Planning,OWP 21/22 -STANISLAUS REGION CONSOLIDATED PLANNING GRANT (CPG) PROJECT FOR STANISLAUS COUNCIL OF GOVERNMENTS (STANCOG) OVERALL WORK PROGRAM (OWP) FOR,1362220.00,0,nan,999,Statewide,Statewide,nan,None,None,Planning and Research in Stanislaus Council Of Governments,"This project is part of the nan Program, and recieved $1362220.0. This project will Planning and Research in Stanislaus Council Of Governments."
4597,2022-12-02,5060369,Fresno,L5060FCOG,Y300,Transportation Alternatives,0620000028L,17|28,Construction Engineering|Facilities for Pedestrians and Bicycles,"FRESNO STREET AND BROWNING AVENUE NEAR ROBINSON ELEMENTARY SCHOOL INSTALL TRAFFIC SIGNAL, PEDESTRIAN COUNTDOWN AND SIGNAL EQUIPMENT, CURB RAMPS, SIGN",504000.00,22,6.0,19,Fresno County,Fresno County,5060.0,Council of Fresno County Governments,Council Of Fresno County Goverments,Install Pedestrian Safety Improvements in Fresno,"This project is part of the Transportation Alternatives Program, and recieved $504000.0. This project will Install Pedestrian Safety Improvements in Fresno."
2785,2022-08-08,5170063,Kingsburg,L5170FCOG,Y400,Congestion Mitigation & Air Quality Improvement,0621000158L,17|28,Construction Engineering|Facilities for Pedestrians and Bicycles,WESTSIDE OF 18TH AVE FROM STROUD AVE TO KLEPPER ST CONSTRUCT NEW SIDEWALKS,120400.00,21,6.0,19,Fresno County,Fresno County,5170.0,Council of Fresno County Governments,Council Of Fresno County Goverments,Construct Sidewalk in Kingsburg,"This project is part of the Congestion Mitigation & Air Quality Improvement Program, and recieved $120400.0. This project will Construct Sidewalk in Kingsburg."
2527,2022-07-20,5937205,Santa Clara County,L5937MTC,Z233,nan,0416000438L,15,Preliminary Engineering,"ALDER CROFT HEIGHTS ROAD OVER HOOKER CREEK, 2.12 MILES SOUTH OF LOS GATOS SC ROAD. BRIDGE NO. 37C0506 REPLACE EXISTING ONE LANE BRIDGE WITH A NEW T",600000.00,18,4.0,85,Santa Clara County,Santa Clara County,5937.0,Metropolitan Transportation Commission,Metropolitan Transportation Commission,Replace Bridge in Santa Clara County,"This project is part of the nan Program, and recieved $600000.0. This project will Replace Bridge in Santa Clara County."
3068,2022-08-29,P020194,California,S SACOG,Z0E1,nan,0313000033S,14|17|42,Bridge Rehabilitation - No Added Capacity|Construction Engineering|Training,IN SUTTER AND YUBA COUNTIES IN YUBA CITY AND MARYSVILLE AT THE FEATHER RIVER BRIDGE AND OVERHEAD. BRIDGE SCOUR MITIGATION,20639700.00,3,3.0,101,Sutter County,Sutter County,nan,None,None,Erosion Counterme

In [10]:
sorted(list(agg.project_title_new))

[' Added Roadway Capacity in Industry',
 ' Added Roadway Capacity in Kern County',
 ' Added Roadway Capacity in Los Angeles',
 ' Added Roadway Capacity in Los Angeles',
 ' Added Roadway Capacity in Los Angeles',
 ' Added Roadway Capacity in Los Angeles',
 ' Added Roadway Capacity in Madera County',
 ' Added Roadway Capacity in Marin County & San Francisco County',
 ' Added Roadway Capacity in Marin County & San Francisco County',
 ' Added Roadway Capacity in Marin County & San Francisco County',
 ' Added Roadway Capacity in Marin County & San Francisco County',
 ' Added Roadway Capacity in Marin County & San Francisco County',
 ' Added Roadway Capacity in Marin County & San Francisco County',
 ' Added Roadway Capacity in Marin County & San Francisco County',
 ' Added Roadway Capacity in Marin County & San Francisco County',
 ' Added Roadway Capacity in Marin County & San Francisco County',
 ' Added Roadway Capacity in Murrieta',
 ' Added Roadway Capacity in Murrieta',
 ' Added Roadway 

In [11]:
len(agg)

5945

In [12]:
agg = _script_utils.title_column_names(agg)

In [19]:
agg.to_csv(f"{GCS_FILE_PATH}/FMIS_Projects_Universe_IIJA_Reporting_rerun_5-29-23_agg.csv")

In [14]:
# ###test full. PASS title, no concat and 1241 rows
# full = _script_utils.get_clean_data(df, full_or_agg = 'full')

In [15]:
# full>>filter(_["Recipient Project Number"]=="0118000055L")

In [16]:
# full = _script_utils.title_column_names(full)

In [20]:
# full.to_csv(f"{GCS_FILE_PATH}/FMIS_Projects_Universe_IIJA_Reporting_rerun_5-29-23_all.csv")

In [18]:
agg

,Fmis Transaction Date,Project Number,Implementing Agency,Summary Recipient Defined Text Field 1 Value,Program Code,Program Code Description,Recipient Project Number,Improvement Type,Improvement Type Description,Project Title,Obligations Amount,Congressional District,District,County Code,County Name,County Name Title,Implementing Agency Locode,Rtpa Name,Mpo Name,Project Title New,New Description Col
0,2022-01-18,0151240,California,S SCAG,Z001,nan,0812000093S,6|17,4R - Restoration & Rehabilitation|Construction Engineering,IN SAN BERNARDINO CO FR 0.3 MI N/O N. ALRAY OVERHEAD TO 0.3 MI S/O OAK HILL RD OH INSTALL VEGETATION CONTROL UNDERNEATH METAL BEAM GUARD RAIL (TC),1202300.56,8,8.0,71,San Bernardino County,San Bernardino County,nan,None,None,Install Overhead in San Bernardino County,"This project is part of the nan Program, and recieved $1202300.56. This project will Install Overhead in San Bernardino County."
1,2022-01-18,15D0001,California,S ER NONE,ER01,Emergency Supplement Funding,0721000066S,6|17,4R - Restoration & Rehabilitation|Construction Engineering,"ON LA-39 PM 18.0/30.9 & 39.5/44.4 IN ANGELES NATIONAL FOREST, IN THE UNINCORPORATED AREA OF COUNTY OF LOS ANGELES EMERGENCY PROJECT - EMERGENCY OPENI",608700.00,27|32,7.0,37,Los Angeles County,Los Angeles County,nan,None,None,Road Restoration & Rehabilitation in Los Angeles County,"This project is part of the Emergency Supplement Funding Program, and recieved $608700.0. This project will Road Restoration & Rehabilitation in Los Angeles County."
2,2022-01-18,32L0015,Santa Cruz County,L5936AMBAG,ER11,nan,0517000116L,15,Preliminary Engineering,IN THE COUNTY OF SANTA CRUZ ON NORTH RODEO GULCH RD PM 4.94 STABILIZE SLOPE AND RECONSTRUCT ROADWAY,94948.00,20,5.0,87,Santa Cruz County,Santa Cruz County,5936.0,Santa Cruz County Regional Transportation Commission,Association of Monterey Bay Area Governments,Road Construction in Santa Cruz County,"This project is part of the nan Program, and recieved $94948.0. This project will Road Construction in Santa Cruz County."
3,2022-01-18,32L0015,Santa Cruz County,L5936AMBAG,ER12,nan,0517000116L,4|17,4R - No Added Capacity|Construction Engineering,IN THE COUNTY OF SANTA CRUZ ON NORTH RODEO GULCH RD PM 4.94 STABILIZE SLOPE AND RECONSTRUCT ROADWAY,459822.83,20,5.0,87,Santa Cruz County,Santa Cruz County,5936.0,Santa Cruz County Regional Transportation Commission,Association of Monterey Bay Area Governments,Road Construction in Santa Cruz County,"This project is part of the nan Program, and recieved $459822.83. This project will Road Construction in Santa Cruz County."
4,2022-01-18,32L0015,Santa Cruz County,L5936AMBAG,ER14,nan,0517000116L,4|15|16|17,4R - No Added Capacity|Preliminary Engineering|Right of Way|Construction Engineering,IN THE COUNTY OF SANTA CRUZ ON NORTH RODEO GULCH RD PM 4.94 STABILIZE SLOPE AND RECONSTRUCT ROADWAY,91381.17,20,5.0,87,Santa Cruz County,Santa Cruz County,5936.0,Santa Cruz County Regional Transportation Commission,Association of Monterey Bay Area Governments,Road Construction in Santa Cruz County,"This project is part of the nan Program, and recieved $91381.17. This project will Road Construction in Santa Cruz County."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5940,2023-03-08,5945103,Kings County,L5945KCAG,Z001,nan,0616000178L,15,Preliminary Engineering,"PM00144 FOR 10 BRIDGES: 45C0006, 45C0015, 45C0033, 45C0044, 45C0081, 45C0095, 45C0096, 45C0101, 45C0113, AND 45C0114. BRIDGE PREVENTATIVE MAINTENANCE",473635.00,21,6.0,31,Kings County,Kings County,5945.0,Kings County Association of Governments - RTPA,Kings County Association of Governments,Preliminary Engineering Projects in Kings County,"This project is part of the nan Program, and recieved $473635.0. This project will Preliminary Engineering Projects in Kings County."
5941,2023-03-08,6801084,California,S MTC,ZS30,nan,0416000148S,17|21|42,Construction Engineering|Safety|Training,"IN AND NEAR WALNUT CREEK, PLEASANT HILL 

In [21]:
len(agg)

5945